#odpy, a computer vision model, built from scratch and implememted into this notebook for experimental purposes. 

A machine learning model is a file that has been trained to recognize certain types of patterns. You train a model over a set of data, providing it an algorithm that it can use to reason over and learn from those data. Once you have trained the model, you can use it to reason over data that it hasn't seen before, and make predictions about that data.



In [ ]:
import mlflow.sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
from mlflow.models.signature import infer_signature
import mlflow
import mlflow.tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import mlflow
import mlflow.tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Set experiment name in MLflow (it will create if it doesn't exist)
EXPERIMENT_NAME = "Tire Defect Detection"
mlflow.set_experiment(EXPERIMENT_NAME)

# Load or generate your data (using random data here for demo)
def load_data():
    # Dummy dataset, replace with actual data loading
    train_images = np.random.rand(100, 224, 224, 3)  # Replace with actual images
    train_labels = np.random.randint(0, 3, (100, 1))  # Replace with actual labels
    return train_images, train_labels

# Example simple CNN model
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(3, activation='softmax')  # Assuming 3 classes (bentRim, goodTire, cracking)
    ])
    return model

# Training and logging the model in MLflow
def train_and_log_model():
    train_images, train_labels = load_data()

    # Prepare the model
    model = build_model()
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # MLflow logging
    with mlflow.start_run() as run:
        print("Running experiment with ID: ", run.info.run_id)

        # Log parameters (you can log hyperparameters)
        mlflow.log_param("optimizer", "adam")
        mlflow.log_param("loss", "sparse_categorical_crossentropy")
        mlflow.log_param("epochs", 5)
        mlflow.log_param("batch_size", 32)

        # Train the model and log training metrics
        history = model.fit(train_images, train_labels, epochs=5, batch_size=32, validation_split=0.2)

        # Log the model
        mlflow.tensorflow.log_model(model, artifact_path="model", registered_model_name="TireDefectModel")

        # Log metrics (training accuracy, loss, etc.)
        for epoch, acc in enumerate(history.history['accuracy']):
            mlflow.log_metric("accuracy", acc, step=epoch)
        for epoch, loss in enumerate(history.history['loss']):
            mlflow.log_metric("loss", loss, step=epoch)

        # Save the final model
        model.save("tire_defect_detection_model.pbtxt")
        print(f"Model saved in run {run.info.run_id}")

if __name__ == "__main__":
    train_and_log_model()



# Start your training job with `start_run()`
with mlflow.start_run() as run:

    lr = LogisticRegression()
    X = np.array([-2, -1, 0, 1, 2, 1]).reshape(-1, 1)
    y = np.array([0, 0, 1, 1, 1, 0])
    lr.fit(X, y)
    score = lr.score(X, y)
    signature = infer_signature(X, y)

    # Activate the MLFlow logging API to log your training job metrics
    print("test log_metrics.")
    mlflow.log_metric("score", score)

    # Activate the MLFlow logging API to log your training job parameters
    print("test log_params.")
    mlflow.log_param("alpha", "alpha")
    print("All done")

c:\Users\garri\odpy\odpy\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Running experiment with ID:  0f593166d6274a6b8e5fbc5305df3965
Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.2641 - loss: 5.5400 - val_accuracy: 0.3500 - val_loss: 1.9117
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.4437 - loss: 1.3212 - val_accuracy: 0.2500 - val_loss: 1.3364
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.3836 - loss: 1.1819 - val_accuracy: 0.2500 - val_loss: 1.1354
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.3680 - loss: 1.0822 - val_accuracy: 0.2500 - val_loss: 1.1956
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.4031 - loss: 1.0834 - val_accuracy: 0.4000 - val_loss: 1.1595


2024/10/27 11:47:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/27 11:48:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'TireDefectModel' already exists. Creating a new version of this model...
Created version '5' of model 'TireDefectModel'.


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=tire_defect_detection_model.pbtxt.

In [6]:
# Import necessary libraries
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Step 1: Load the model
model_path = "odpy/mlruns/models/TireDefectModel/version-1/meta.yaml"  # Update with your model's path
model = tf.saved_model.load(model_path)
print("Model loaded successfully.")

# Step 2: Load and preprocess the test image
def load_image(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (640, 640))  # Resize to model's input size
    img_array = np.array(img_resized) / 255.0  # Normalize
    return img_rgb, img_array[np.newaxis, ...]  # Add batch dimension

image_path = "odpy/images/bentRim.jpg"  # Update with your test image path
original_img, preprocessed_img = load_image(image_path)
print("Test image loaded and preprocessed.")

# Step 3: Make predictions
def get_predictions(model, image):
    outputs = model(image)
    predictions = outputs['detection_boxes'][0].numpy()
    scores = outputs['detection_scores'][0].numpy()
    classes = outputs['detection_classes'][0].numpy().astype(int)
    return predictions, scores, classes

predictions, scores, classes = get_predictions(model, preprocessed_img)

# Step 4: Filter predictions by confidence threshold
confidence_threshold = 0.5
filtered_boxes = []
filtered_labels = []

for i, score in enumerate(scores):
    if score >= confidence_threshold:
        box = predictions[i]
        filtered_boxes.append(box)
        filtered_labels.append(classes[i])  # Replace with label names if available
print(f"Filtered {len(filtered_boxes)} boxes with confidence >= {confidence_threshold}")

# Step 5: Visualize predictions
def draw_boxes(image, boxes, labels):
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    for box, label in zip(boxes, labels):
        ymin, xmin, ymax, xmax = box
        rect = patches.Rectangle(
            (xmin * image.shape[1], ymin * image.shape[0]),
            (xmax - xmin) * image.shape[1],
            (ymax - ymin) * image.shape[0],
            linewidth=2, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(xmin * image.shape[1], ymin * image.shape[0] - 10, str(label), color='red', fontsize=12)
    
    plt.axis('off')
    plt.show()

draw_boxes(original_img, filtered_boxes, filtered_labels)



OSError: SavedModel file does not exist at: odpy/mlruns/models/TireDefectModel/version-1/meta.yaml\{saved_model.pbtxt|saved_model.pb}